Training notebook for [Diff-SVC](https://github.com/prophesier/diff-svc) originally made by [justinjohn-03](https://github.com/justinjohn0306)
 Modified by [奕晨](https://twitter.com/nekrothecorpse) of [Archivoice](https://github.com/archivoice) and currently maintained by [haru0l](https://twitter.com/mscoocoo2)

Please don't use this thing for illegal stuff like copying the voices of celebs and such. Just make sure you have permission!

Private ver of the notebook for the Diff-SVC server. No redistribution allowed! No saving either.

Support us over on Ko-Fi!

[haru0l](https://ko-fi.com/haru0l)

[Kangarroar](https://ko-fi.com/kangarroar)

[julieraptor](https://ko-fi.com/julieraptor)

[MLo7](https://ko-fi.com/m_lo7)

[justinjohn-03](https://ko-fi.com/justinjohn03)

# Check Setup

# **Kanga Fart Edition ver 2.2f**

In [ ]:
#@title #Check GPU type
#@markdown this is for checking the GPU type you have as well as the available amount of vram.
!nvidia-smi -L
!nvidia-smi

GPU 0: Tesla T4 (UUID: GPU-be34d154-6aa3-e5a5-a76d-62d6dc1e3c85)
Thu Aug  3 08:09:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                       

In [ ]:
#@title #Mount Google Drive

#@markdown Makes your life easier when uploading and saving stuff.

from google.colab import drive
drive.flush_and_unmount()
!rm -rf /content/drive
drive.mount('/content/drive')
print('Done!')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
Done!


# Preparation

In [ ]:
from IPython.display import clear_output
import time
import os
#if this notebook gets stolen again im quitting diffsvc support
#from haru0l (25/3/2023)
#@title #Step 1: Setup (part 1)

#@markdown 【TERMS OF USE】

#@markdown Disclaimer:

#@markdown By using this notebook, you agree that you will NOT clone a person's voice WITHOUT express permission and you may NOT monetize from this.

#@markdown The authors of this notebook, neither will the developers will NOT assume responsibility for YOUR action for using this project. Only YOU as the user will take responsibility for your action.

#@markdown You are also forbidden to download or share the notebook to anyone.

#@markdown If you agree with the terms above, feel free to use the notebook

#@markdown If I see you training a celebrity voice on here I'm not gonna keeping this notebook online and provide support.

agree = True #@param {type: "boolean"}

if agree:
    !rm -rf sample_data
    !python -m pip install --upgrade pip wheel --quiet
    !pip uninstall gdown -y --quiet
    #!pip install git+https://github.com/justinjohn0306/gdown.git --quiet
    !pip install pydub fuzzywuzzy python-Levenshtein
    !pip install torch==1.13.1 torchvision torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117

    #install aria2
    !sudo apt-get install aria2  &> /dev/null
    !apt install wget curl ca-certificates &> /dev/null
    !wget -N git.io/aria2.sh &> /dev/null && chmod +x aria2.sh &> /dev/null
    !echo 1|./aria2.sh &> /dev/null
    !echo 12|./aria2.sh &> /dev/null
    !echo 6|./aria2.sh &> /dev/null
    !pip install --pre torchtext==0.6.0 --no-deps --quiet

    #@markdown ###Select which fork to use
    #@markdown The official repo is up-to-date, while UtaUtaUtau's version has Harvest support for improved f0.
    fork = "UtaUtaUtau's Repo" #@param ["Official Diff-SVC Repo", "UtaUtaUtau's Repo"]

    print('Installing Diff-SVC')
    if fork == "Official Diff-SVC Repo":
        !git clone -q https://github.com/prophesier/diff-svc
    else:
        !git clone -q --branch harvest-preprocess https://github.com/UtaUtaUtau/diff-svc
        #!pip install pyworld==0.3.1
    %cd "diff-svc"
    !pip install -r requirements_short.txt --quiet
    !pip install --upgrade numba==0.56.3

    clear_output()
    exit()
    #if you see this in the request-help section, gaslight the user
    raise Exception('Restarting notebook... continue as usual. This is normal. your notebook has not crashed please just continue STOP ASKING ABOUT THIS ITS FINE THIS IS NOT AN ERRORRRRRRR')
    #if you see this say fart

else:
    print("So you did not agree...")

Exception: ignored

In [ ]:
#@markdown ## Setup (part 2)
from IPython.display import clear_output
from IPython.display import Audio
from IPython.display import display
from IPython.display import Image

#@markdown ###Model sample rate
#@markdown Please choose if you want to train a 24kHz model or a 44.1kHz model.

sample_rate = '44.1kHz' #@param ["24kHz", "44.1kHz"]
%cd "diff-svc"
!pip install tensorboard<2.9,>=2.8 --quiet
%reload_ext tensorboard
%mkdir -p checkpoints

hifigan_24k = "https://github.com/haru0l/Diff-SVC-notebooks/releases/download/models_24khz/hifigan_24k.zip"
hifigan_44k = "https://github.com/haru0l/Diff-SVC-notebooks/releases/download/start/hifigan_44k.zip"
checkpoints = "https://github.com/haru0l/Diff-SVC-notebooks/releases/download/start/checkpoints.zip"

!aria2c --file-allocation=none -c -x 10 -s 10 {checkpoints} -q
!unzip checkpoints.zip
!rm checkpoints.zip

if sample_rate == "44.1kHz":
    !aria2c --file-allocation=none -c -x 10 -s 10 {hifigan_44k} -q
    !unzip hifigan_44k.zip -d checkpoints
    !rm hifigan_44k.zip
    config_path = "/content/diff-svc/training/config_nsf.yaml"
    slay = "44100"
else:
    !aria2c --file-allocation=none -c -x 10 -s 10 {hifigan_24k} -q
    !unzip hifigan_24k.zip
    !rm hifigan_24k.zip
    config_path = "/content/diff-svc/training/config.yaml"
    !rm {config_path}
    !wget "https://github.com/haru0l/Diff-SVC-notebooks/releases/download/models_24khz/config.yaml" -O {config_path} -q
    slay = "24000"

setup_complete=True
#!pip install pyworld==0.3.1
#clear_output()

In [ ]:
import os
import subprocess
from pydub import AudioSegment
import csv
from fuzzywuzzy import fuzz
import re
from IPython.display import clear_output
import wave
import shutil
%cd "/content/diff-svc"
#@title #Step 2: Decompress dataset
#@markdown This should work with most common archive formats, so don't worry. Please stick with the alphabet characters otherwise preprocessing might error out.

#@markdown Supported types: `.rar`, `.zip`, `.tar`, `.tar.gz`, `.tar.bz2`, `.7z`

#@markdown ###Note that your dataset should consist of `.wav` or `.ogg` format audio
#@markdown ---
#@markdown Name your singer.
singer_name = 'stinky' #@param {type: "string"}

#@markdown ---
#@markdown ###Cleanup folders
#@markdown Checking this will delete the folder in case you made any errors and need to rerun this cell.
cleanup = True #@param {type: "boolean"}

#@markdown ---
#@markdown ###Attempt to increase range
#@markdown This uses UtaUtaUtau's script using PyWORLD to artificially add range to your model. It doesn't sound very natural though. And it's slow.
increase_range = False #@param {type: "boolean"}

if cleanup:
  !rm -rf "data/raw"


if singer_name == '' or singer_name.replace(" ", "") == '' or singer_name.replace(" ", "") == '':
  print("Put a singer name.")
else:
  singer_name = re.sub(r'[àáâãäå]', 'a', singer_name)
  singer_name = re.sub(r'[èéêë]', 'e', singer_name)
  singer_name = re.sub(r'[ìíîï]', 'i', singer_name)
  singer_name = re.sub(r'[òóôõö]', 'o', singer_name)
  singer_name = re.sub(r'[ùúûü]', 'u', singer_name)
  singer_name = re.sub(r'[ýÿ]', 'y', singer_name)
  singer_name = re.sub(r'[ÀÁÂÃÄÅ]', 'A', singer_name)
  singer_name = re.sub(r'[ÈÉÊË]', 'E', singer_name)
  singer_name = re.sub(r'[ÌÍÎÏ]', 'I', singer_name)
  singer_name = re.sub(r'[ÒÓÔÕÖ]', 'O', singer_name)
  singer_name = re.sub(r'[ÙÚÛÜ]', 'U', singer_name)
  singer_name = re.sub(r'[Ý]', 'Y', singer_name)
  singer_name = re.sub(r'[ñ]', 'n', singer_name)

  singer_name = singer_name.replace(" ", "_")
  clear_output()
  if sample_rate == "44.1kHz":
      !sed -i -r 's/nyaru/{singer_name}/g' {config_path}
  else:
      !sed -i -r 's/atri/{singer_name}/g' {config_path}

  #@markdown ---
  #@markdown File location
  !mkdir -p data/raw
  dataset_location = '' #@param {type: "string"}
  diffsvc_location = os.path.join('data', 'raw', singer_name, "")
  if dataset_location == '' or dataset_location.replace(" ", "") == '' or dataset_location.replace(" ", "") == '':
    print("No dataset loaded. You have not specified a path.")
  else:
    if dataset_location.endswith('.rar'):
        !unrar x "$dataset_location" "$diffsvc_location"
    elif dataset_location.endswith('.zip'):
        !unzip "$dataset_location" -d "$diffsvc_location"
    elif dataset_location.endswith('.tar'):
        !tar -xf "$dataset_location" -C "$diffsvc_location"
    elif dataset_location.endswith('.tar.gz'):
        !tar -xzf "$dataset_location" -C "$diffsvc_location"
    elif dataset_location.endswith('.tar.bz2'):
        !tar -xjf "$dataset_location" -C "$diffsvc_location"
    else:
        !7za x "$dataset_location" -o$diffsvc_location

    root_folder = diffsvc_location

    # create a list to store the converted audio files
    converted_files = []

    # loop through all subfolders and files in the root folder
    for subdir, dirs, files in os.walk(root_folder):
        for file in files:
            # get the file path
            file_path = os.path.join(subdir, file)
            # check if the file is an audio file
            if file.endswith('.mp3') or file.endswith('.wav') or file.endswith('.flac') or file.endswith('.ogg'):
                try:
                    # load the audio file
                    audio = AudioSegment.from_file(file_path)
                    # check if the file is shorter than 3 seconds
                    if audio.duration_seconds < 3:
                        os.remove(file_path)
                    else:
                        # convert the audio file to WAV format
                        audio = audio.set_frame_rate(44100).set_sample_width(2).set_channels(1)
                        new_file_path = os.path.join(subdir, 'wav_' + str(len(converted_files) + 1) + '.wav')
                        audio.export(new_file_path, format='wav')
                        # add the new file to the list of converted files
                        converted_files.append(new_file_path)
                        # remove the original file
                        os.remove(file_path)
                except Exception as e:
                    print(f'Error converting {file_path} : {e}')
                    os.remove(file_path)
            else:
                os.remove(file_path)

    if increase_range:
      %cd {root_folder}
      !wget https://cdn.discordapp.com/attachments/1044948982977286186/1067717271671291904/ZkSFJNA.py --quiet
      !python ZkSFJNA.py
      !rm ZkSFJNA.py
      %cd ../../../
    print("Done!")

  #clear_output()

In [ ]:
import os
import subprocess
from pydub import AudioSegment
import csv
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import yaml
from IPython.display import clear_output
import re

#@title #Step 2-A: Decompress training data
#@markdown Decompresses training data directly to `diff-svc/data/binary`, usable only if you already have the output files of Step 4. The files from this should be in your Google Drive.

#@markdown If you run this step, please skip step 4.
%cd "/content/diff-svc"
#@markdown ---
#@markdown File location
!mkdir -p data/binary
preprocessed_data_location = '' #@param {type: "string"}
config_location = '' #@param {type: "string"}
diffsvc_bin_location = 'data'

os.environ['PYTHONPATH']='.'
with open(config_location, 'r') as config_file:
    config = yaml.safe_load(config_file)
    singer_name = config['speaker_id']

singer_name = re.sub(r'[àáâãäå]', 'a', singer_name)
singer_name = re.sub(r'[èéêë]', 'e', singer_name)
singer_name = re.sub(r'[ìíîï]', 'i', singer_name)
singer_name = re.sub(r'[òóôõö]', 'o', singer_name)
singer_name = re.sub(r'[ùúûü]', 'u', singer_name)
singer_name = re.sub(r'[ýÿ]', 'y', singer_name)
singer_name = re.sub(r'[ÀÁÂÃÄÅ]', 'A', singer_name)
singer_name = re.sub(r'[ÈÉÊË]', 'E', singer_name)
singer_name = re.sub(r'[ÌÍÎÏ]', 'I', singer_name)
singer_name = re.sub(r'[ÒÓÔÕÖ]', 'O', singer_name)
singer_name = re.sub(r'[ÙÚÛÜ]', 'U', singer_name)
singer_name = re.sub(r'[Ý]', 'Y', singer_name)
singer_name = re.sub(r'[ñ]', 'n', singer_name)

#@markdown ---
#@markdown ###Cleanup folders
#@markdown Checking this will delete the folder in case you made any errors and need to rerun this cell.
cleanup = False #@param {type: "boolean"}

if cleanup:
  !rm -rf "data/binary"

print("Processing....")


singer_name = singer_name.replace(" ", "_")
clear_output()
if sample_rate == "44.1kHz":
  !sed -i -r 's/nyaru/{singer_name}/g' {config_path}
else:
  !sed -i -r 's/atri/{singer_name}/g' {config_path}

if preprocessed_data_location == '' or preprocessed_data_location.replace(" ", "") == '' or preprocessed_data_location.replace(" ", "") == '':
  print("No dataset loaded. You have not specified a path.")
else:
  if preprocessed_data_location.endswith('.rar'):
      !unrar x "$preprocessed_data_location" "$diffsvc_bin_location"
  elif preprocessed_data_location.endswith('.zip'):
      !unzip "$preprocessed_data_location" -d "$diffsvc_bin_location"
  elif preprocessed_data_location.endswith('.tar'):
      !tar -xf "$preprocessed_data_location" -C "$diffsvc_bin_location"
  elif preprocessed_data_location.endswith('.tar.gz'):
      !tar -xzf "$preprocessed_data_location" -C "$diffsvc_bin_location"
  elif preprocessed_data_location.endswith('.tar.bz2'):
      !tar -xjf "$preprocessed_data_location" -C "$diffsvc_bin_location"
  else:
      !7za x "$preprocessed_data_location" -o$diffsvc_bin_location

if config_location == '' or config_location.replace(" ", "") == '' or config_location.replace(" ", "") == '':
  print("No config loaded. You have not specified a path.")
else:
  if sample_rate == "44.1kHz":
    !cp -r {config_location} training

print("Done!")

# Step 3: Training Options/Parameters
Unfortunately, you can not get away with no editing, not completely that is.

In [ ]:
%cd /content/diff-svc

import re
import os
os.environ['PYTHONPATH']='.'
with open(config_path, "r") as f:
    yaml_file = f.read()

yaml_file = re.sub(r'load_ckpt: \S+', 'load_ckpt: ""', yaml_file)

with open(config_path, "w") as f:
    f.write(yaml_file)

#@markdown ###F0 extraction method
#@markdown Crepe is used for F0 extraction for data preprocessing, while it is of higher quality, it is slow, therefore set to false as default.

#@markdown Unchecking this while using the official repo will default to parselmouth, while using UtaUtaUtau's repo will use harvest.
use_crepe = True #@param {type: "boolean"}

#@markdown ---
#@markdown ###Use custom save directory
#@markdown You can change the directory to save wherever you want. Default location is /diff-svc/checkpoint if unchanged.

#@markdown Please point to a directory with the singer name already specified (example /content/drive/MyDrive/diff-svc/nyaru)

use_save_dir = False #@param {type: "boolean"}

save_dir = "checkpoints" #@param {type: "string"}
#save_dir.strip() check for issues, disabling temporarily

#@markdown ---
#@markdown ###Resume training from a checkpoint

resume_training_from_ckpt = False #@param {type: "boolean"}
ckpt_directory = "" #@param {type: "string"}

#@markdown ---

#@markdown ###Setup for small datasets
#@markdown If your dataset is small, each epoch will go by very fast and won't have enough time to train well, so if your dataset is considered small, use this option.

endless_ds = False #@param {type:"boolean"}

if use_save_dir:
  !rm -rf utils/hparams.py
  !wget https://github.com/prophesier/diff-svc/raw/main/utils/hparams.py -O utils/hparams.py
  !sed -i -r 's|checkpoints/\{args.work_dir}|haruprivatewashere|g' /content/diff-svc/utils/hparams.py
  !sed -i -r 's|haruprivatewashere|{save_dir}|g' /content/diff-svc/utils/hparams.py
else:
  !rm -rf utils/hparams.py
  !wget https://github.com/prophesier/diff-svc/raw/main/utils/hparams.py -O utils/hparams.py
clear_output()

if resume_training_from_ckpt:
  !cp {ckpt_directory} {save_dir}
  print("Status: Resuming Training From Checkpoint")
else:
  print("Status: Not Resuming From Checkpoint")

batch_size = 12
if sample_rate == "44.1kHz":
  decay_steps = 40000
  disable_fs2 = True
else:
  decay_steps = 60000
  disable_fs2 = False

!sed -i -r 's|(max_sentences:)(\s+)(.+)|\1\2{batch_size}|g' {config_path}
!sed -i -r 's|(decay_steps:)(\s+)(.+)|\1\2{decay_steps}|g' {config_path}
!sed -i -r 's|(use_crepe:)(\s+)(.+)|\1\2{use_crepe}|g' {config_path}
!sed -i -r 's|(no_fs2:)(\s+)(.+)|\1\2{disable_fs2}|g' {config_path}
!sed -i -r 's|(endless_ds:)(\s+)(.+)|\1\2{endless_ds}|g' {config_path}

#clear_output()

# Training
Finally, the dreaded part.

In [ ]:
#@title #Step 4: Pre-processing
#@markdown This step is also known as data prep or feature generation, who cares?
#@markdown

os.environ['PYTHONPATH']='.'
!CUDA_VISIBLE_DEVICES=0 python preprocessing/binarize.py --config {config_path}
%cd "data"
!7za -bso0 a "../../{singer_name}_binary_data.7z" "binary/{singer_name}"

#@markdown ---
#@markdown ###Save binary to Google Drive

#prolly remove this for porting this notebook to other platforms
save_binary_to_gd = False #@param {type: "boolean"}

if save_binary_to_gd:
  if not os.path.exists('/content/drive/MyDrive/diff-svc/data/{singer_name}'):
      !mkdir -p /content/drive/MyDrive/diff-svc/data/{singer_name}
  !mv -v "../../{singer_name}_binary_data.7z" /content/drive/MyDrive/diff-svc/data/{singer_name}
  !cp -r {config_path} /content/drive/MyDrive/diff-svc/data/{singer_name}
else:
  !cp -r {config_path} ../../
%cd ../

In [ ]:
#@title #Step 5-0 Tensorboard (run before step 5)
#@markdown Shows training progress, go to the top right corner to set it to update the logs.

#@markdown ---
#@markdown ### 403 Forbidden error fix

#@markdown You're better off [checking your cookie settings to allow colab and google servers](https://stackoverflow.com/questions/64218755/getting-error-403-in-google-colab-with-tensorboard-with-firefox).
fix_403 = False #@param {type: "boolean"}
%cd "/content/diff-svc"
import datetime, os
%reload_ext tensorboard
if use_save_dir:
  %tensorboard --reload_interval=1 --reload_multifile=true --logdir="{save_dir}/lightning_logs/"
elif use_save_dir and fix_403:
  %tensorboard --reload_interval=1 --reload_multifile=true --logdir="{save_dir}/lightning_logs/" --bind_all
elif fix_403:
  %tensorboard --reload_interval=1 --reload_multifile=true --logdir=checkpoints/{singer_name}/lightning_logs/ --bind_all
else:
  %tensorboard --reload_interval=1 --reload_multifile=true --logdir=checkpoints/{singer_name}/lightning_logs/

In [ ]:
import re
import yaml
%cd "/content/diff-svc"
#@title #Step 5: Training
#@markdown Yeah, it took THAT long to get here, colab is probably going to disconnect you at this point... unless you have pro ¯\\_(ツ)_/¯

#@markdown ---
#@markdown ### General torch errors (untested)

#@markdown Check if you have this error
fix_torch = False #@param {type: "boolean"}
os.environ['PYTHONPATH']='.'
with open(config_path, 'r') as config_file:
    config = yaml.safe_load(config_file)
    config_sample_rate = config['audio_sample_rate']

if slay == str(config_sample_rate):
  if fix_torch:
    os.environ['PYTHONPATH']='.'
    !CUDA_VISIBLE_DEVICES=0 python preprocessing/binarize.py --config {config_path}
    !CUDA_VISIBLE_DEVICES=0 python run.py --config {config_path} --exp_name $singer_name --reset
  else:
    os.environ['PYTHONPATH']='.'
    !CUDA_VISIBLE_DEVICES=0 python run.py --config {config_path} --exp_name $singer_name --reset
else:
  print(f"Hold on, you're trying to train on the {slay} config but your config is using the {config_sample_rate}. Please restart the notebook using {config_sample_rate}.")

In [ ]:
#@title # Step 6: Package Model (unused)
#@markdown If you used a custom directory for saving your models please don't use this.

store_on_drive = True #@param {type: "boolean"}
import glob

%cd /content/diff-svc/checkpoints
!zip -r "/content/{archive_name}.zip" ./{singer_name} -x ./{singer_name}/lighting_logs/\*

if store_on_drive:
      if not os.path.exists('/content/drive/MyDrive/Diff-SVC_release'):
          !mkdir /content/drive/MyDrive/Diff-SVC_release

      !mv -v "/content/{archive_name}.zip" /content/drive/MyDrive/Diff-SVC_release

print('Done!')

# **Inference Section**

mmhmm
yass support us! we're dying. send us money now.

[haru0l](https://ko-fi.com/haru0l)

[Kangarroar](https://ko-fi.com/kangarroar)

[julieraptor](https://ko-fi.com/julieraptor)

[MLo7](https://ko-fi.com/m_lo7)

[justinjohn-03](https://ko-fi.com/justinjohn03)

In [ ]:
import os
import yaml
%cd "/content/diff-svc"
#@title # **Load model**

#@markdown ### **Load a trained model for inferencing**
#@markdown ___

os.environ['PYTHONPATH']='.'
!CUDA_VISIBLE_DEVICES=0

from utils.hparams import hparams
#latest confirmed
from preprocessing.data_gen_utils import get_pitch_parselmouth,get_pitch_crepe
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import utils
import librosa
import torchcrepe
from infer import *
import logging
from infer_tools.infer_tool import *

logging.getLogger('numba').setLevel(logging.WARNING)

model_path = "" #@param {type: "string"}
project_name = "sample"
config_path= "" #@param {type: "string"}
hubert_gpu=True
svc_model = Svc(project_name,config_path,hubert_gpu, model_path)
print('model loaded')

#@title Run Inference (yes I'm keeping this image here incase someone want to kidly buy our ko-fi <3)

#@markdown put your wav paths in here
wav_in = "" #@param {type:"string"}

if wav_in == "":
  print("No input given. Using default audio for reference")
  wav_in = "/content/diff-svc/raw/test_input.wav"
key = 0 #@param {type:"slider", min:-12, max:12, step:1}

pndm_speedup = 10 #@param {type:"slider", min:0, max:100, step:1}

wav_out = "sample"

add_noise_step = 0 #@param {type:"slider", min:0, max:1000, step:10}

thre = 0.85 #@param {type:"slider", min:0.0, max:1.0, step:0.01}
use_crepe= False #@param {type:"boolean"}
use_pe=False #@param {type:"boolean"}
use_gt_mel= False

try:
  f0_tst, f0_pred, audio = run_clip(svc_model,file_path=wav_in, key=key, acc=pndm_speedup, use_crepe=use_crepe, use_pe=use_pe, thre=thre, use_gt_mel=use_gt_mel, add_noise_step=add_noise_step,project_name=project_name,out_path=f"results/{wav_out}.flac")
  clear_output()
  print("Inference complete!")
  print(f"Inferenced file is stored at results/{wav_out}.flac.")
except Exception as e:
  raise Exception(e)


In [ ]:
#@markdown Show results

import os
if os.path.isfile('results/sample.flac'):
  print("slay mama")
else:
  raise Exception("file not found you dumbo")
from IPython.display import Audio
Audio('results/sample.flac')

![](https://cdn.discordapp.com/attachments/1067182751171756033/1090333432128745602/IMG_6220.png)